In [2]:
import pandas as pd
import numpy as np
import osmnx as ox
import datetime
import seaborn as sn
from shapely import wkb
from shapely.geometry import Point
from shapely.geometry import LineString
import geopandas as gp
import matplotlib.pyplot as plt
import random, string
from scipy.spatial.distance import cdist
import os, sys
%matplotlib inline
from collections import deque
import networkx as nx
from time import sleep
from copy import deepcopy
from sympy import Matrix, init_printing
import pickle
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import johnson
import pandas as pd
from itertools import tee
from matplotlib.pyplot import imread

In [3]:
trip_filename = 'data/gps-csv/51513535/part-00000-498c167d-fb29-4098-a9f9-682631b98b93-c000.csv'
temp_trip = pd.read_csv(trip_filename, delimiter=',', header=0)

In [4]:
temp_trip

,route_id,longitude,latitude,timestamp,bearing
0,0,51.304943,35.753280,2020-07-19 11:31:55.112000,270.75690
1,0,51.304443,35.753290,2020-07-19 11:31:57.115000,270.75690
2,0,51.304320,35.753290,2020-07-19 11:31:57.620000,270.75690
3,1,51.304966,35.753280,2020-07-19 23:04:40.246000,270.94610
4,1,51.304745,35.753284,2020-07-19 23:04:41.261000,270.94610
...,...,...,...,...,...
2108,433,51.308304,35.746666,2020-07-19 21:54:18.730000,85.03813
2109,433,51.308304,35.746666,2020-07-19 21:54:20.351000,85.03813
2110,433,51.308304,35.746666,2020-07-19 21:54:24.640000,85.03813
2111,433,51.308304,35.746666,2020-07-19 21:54:25.167000,85.03813


In [11]:
image_path = 'gis12_mapinference/test.png'
# image = imread(image_path)
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

In [9]:
image_bit_depth = int((image.dtype.itemsize * 8) / 2)

In [11]:
image_thresholds = [2**x for x in range(image_bit_depth, 3, -1)] + list(range(15, 0, -1))

In [12]:
image_thresholds

[65536,
 32768,
 16384,
 8192,
 4096,
 2048,
 1024,
 512,
 256,
 128,
 64,
 32,
 16,
 15,
 14,
 13,
 12,
 11,
 10,
 9,
 8,
 7,
 6,
 5,
 4,
 3,
 2,
 1]

In [13]:
[2**x for x in range(image_bit_depth, 3, -1)]

[65536, 32768, 16384, 8192, 4096, 2048, 1024, 512, 256, 128, 64, 32, 16]

In [2]:
import cv2
import numpy as np

In [3]:
height = 200
width = 350
temp = np.zeros((height, width), np.uint8)
# temp16 = np.zeros((height, width), np.uint16)
themap = np.zeros((height, width), np.uint16)

In [4]:
temp[-2, -2] = 270
temp[-1, -1] = 150
temp[-3, -3] = 20

In [5]:
temp

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,  20,   0,   0],
       [  0,   0,   0, ...,   0,  14,   0],
       [  0,   0,   0, ...,   0,   0, 150]], dtype=uint8)

In [6]:
temp16 = np.uint16(temp)

In [7]:
temp16

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,  20,   0,   0],
       [  0,   0,   0, ...,   0,  14,   0],
       [  0,   0,   0, ...,   0,   0, 150]], dtype=uint16)

In [8]:
themap

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [9]:
themap = cv2.add(themap, temp16, themap)

In [10]:
themap

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,  20,   0,   0],
       [  0,   0,   0, ...,   0,  14,   0],
       [  0,   0,   0, ...,   0,   0, 150]], dtype=uint16)

In [46]:
cv2.Line(temp, (50, 60), (100, 150), (32), 1, cv2.cv2_AA)

AttributeError: module 'cv2' has no attribute 'Line'

In [51]:
temp16

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0, 270,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint16)

In [52]:
temp

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0, 14,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=uint8)

In [54]:
temp16 = np.uint8(temp16)

In [42]:
cv2.convertScaleAbs(temp, temp16, 1,0)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 2, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [55]:
temp16

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0, 14,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=uint8)

In [59]:
img.shape

(685, 718)

In [61]:
img[300]

array([255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 254, 253, 246, 249, 255, 255, 145,
        86, 149, 219, 253, 247, 145, 121, 154, 231, 165,  69,  38, 145,
       233, 249, 253, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 250,
       250, 254, 252, 253, 254, 253, 254, 254, 254, 221, 153,  3

In [14]:
cv2.line(img, (150, 450), (400, 600), 32, thickness=5)
cv2.imwrite('test1.png', img)

True

In [15]:
from scipy.ndimage.morphology import grey_closing

In [17]:
np.zeros_like

<function numpy.zeros_like(a, dtype=None, order='K', subok=True, shape=None)>

In [18]:
import scipy.ndimage as nd

In [21]:
np.mgrid

In [22]:
import math

In [23]:
math.fmod

<function math.fmod(x, y, /)>